In [1]:
## Imports
from __future__ import print_function

import torch
from torch import optim
from torch.utils.data import DataLoader
import torch.nn as nn

from unet.unet_model import UNet

from utils.dataset import CrossValDataset
from utils.model_eval import dice_coefficient

from torch.utils.tensorboard import SummaryWriter

In [2]:
## Device
if torch.cuda.is_available():
    device = 'cuda'
    print('Using GPU')
else:
    device = 'cpu'
    print('Using CPU')

Using GPU


In [3]:
## Dataset
groupPath = "D:\\autoseg\\train_spine"
batch_size = 8

## Training
n_channels = 1
n_classes = 14

learning_rate = 1e-3
weight_decay  = 1e-8
momentum      = 0.9

num_epochs    = 81

In [4]:
## Train
Npartition = 5
for partition in range(5):
    trainDataset = CrossValDataset(groupPath,partition, testFlag = False)
    testDataset  = CrossValDataset(groupPath,partition, testFlag = True)
    trainLoader  = DataLoader(trainDataset,batch_size=batch_size)
    testLoader   = DataLoader(testDataset,batch_size=batch_size)
    
    # Define Net
    net = UNet(n_channels=n_channels,n_classes=n_classes)
    net.to(device=device)
    
    # Optimizer
    optimizer = optim.RMSprop(net.parameters(),lr=learning_rate,weight_decay=weight_decay,momentum=momentum)
    critereon = nn.CrossEntropyLoss()
    
    #Tensorboard
    writer = SummaryWriter('runs/full_model_spine_partition_'+str(partition))
    
    for epoch in range(num_epochs):
        net.train()
        epoch_train_loss = 0
        for trainIndex,batch in enumerate(trainLoader,0):
            imgs    = batch['img']
            segs_gt = batch['seg']
            
            imgs    = imgs.to(   device=device,dtype=torch.float32)
            segs_gt = segs_gt.to(device=device,dtype=torch.long)
            
            segs_pr = net(imgs)
            
            loss = critereon(segs_pr,segs_gt)
            epoch_train_loss += loss.item()/len(trainLoader)
            
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_value_(net.parameters(),0.1)
            optimizer.step()
            
            print("Train: " + str(loss.item()))
            
        writer.add_scalar('Training Loss',
                        epoch_train_loss,
                        epoch)
        
        net.eval()
        epoch_test_loss = 0
        for testIndex,batch in enumerate(testLoader,0):
            imgs    = batch['img']
            segs_gt = batch['seg']
            
            imgs    = imgs.to(   device=device,dtype=torch.float32)
            segs_gt = segs_gt.to(device=device,dtype=torch.long)
            
            segs_pr = net(imgs)
            
            loss = critereon(segs_pr,segs_gt)
            epoch_test_loss += loss.item()/len(testLoader)
            
            print("Test: " + str(loss.item()))
            
        writer.add_scalar('Validation Loss',
                        epoch_test_loss,
                        epoch)
        
        savefile = "D:\\autoseg\\nets\\unet-spine-save" + str(partition) + "_" + str(epoch) + ".pth"
        torch.save(net.state_dict(),savefile)

Indexing data directory: D:\autoseg\train_spine...
5 groups found.
Partition selected: 0
Serving training data.
Indexing data directory: D:\autoseg\train_spine...
5 groups found.
Partition selected: 0
Serving test data.
Train: 2.7144594192504883
Train: 2.240459442138672
Train: 1.4906467199325562
Train: 1.0718480348587036
Train: 0.7804104089736938
Train: 0.7892652750015259
Train: 0.6935420632362366
Train: 0.638613224029541
Train: 0.6524608731269836
Train: 0.6200339198112488
Train: 0.5712109208106995
Train: 0.5473728775978088
Train: 0.5317412614822388
Train: 0.48848891258239746
Train: 0.4712081849575043
Train: 0.47291192412376404
Train: 0.43959179520606995
Train: 0.4001244008541107
Train: 0.39324823021888733
Train: 0.37766340374946594
Train: 0.45772629976272583
Train: 0.3857284188270569
Train: 0.3770228326320648
Train: 0.3557378649711609
Train: 0.36921030282974243
Train: 0.34029996395111084
Train: 0.321054607629776
Test: 14.37116813659668
Test: 14.91238021850586
Test: 12.728914260864258


Test: 0.679470419883728
Test: 0.3702581524848938
Test: 0.4786469638347626
Train: 0.08214744925498962
Train: 0.06997614353895187
Train: 0.06509637087583542
Train: 0.07301972806453705
Train: 0.0606977604329586
Train: 0.07070142775774002
Train: 0.07647519558668137
Train: 0.07201909273862839
Train: 0.06592364609241486
Train: 0.06256815046072006
Train: 0.05428113415837288
Train: 0.05574919655919075
Train: 0.05664801597595215
Train: 0.05112094804644585
Train: 0.06318805366754532
Train: 0.07121803611516953
Train: 0.06442689895629883
Train: 0.05978374183177948
Train: 0.055252980440855026
Train: 0.0522233210504055
Train: 0.08780518174171448
Train: 0.08017535507678986
Train: 0.0658431276679039
Train: 0.10791686177253723
Train: 0.07263358682394028
Train: 0.06965865194797516
Train: 0.07173392921686172
Test: 0.44886425137519836
Test: 0.8184272050857544
Test: 0.8023782968521118
Test: 0.12829144299030304
Test: 0.8038864135742188
Test: 0.4668154716491699
Test: 0.587302029132843
Train: 0.08076874166727

Test: 0.43274974822998047
Test: 0.5573467016220093
Train: 0.0481148362159729
Train: 0.04445068538188934
Train: 0.0494195818901062
Train: 0.046104613691568375
Train: 0.045653969049453735
Train: 0.05062088370323181
Train: 0.043972115963697433
Train: 0.05031280964612961
Train: 0.04031123220920563
Train: 0.04188564419746399
Train: 0.03943596035242081
Train: 0.03913939744234085
Train: 0.03498869761824608
Train: 0.03294438496232033
Train: 0.044398970901966095
Train: 0.03764639422297478
Train: 0.03871411457657814
Train: 0.028665469959378242
Train: 0.040207743644714355
Train: 0.03332164138555527
Train: 0.054128412157297134
Train: 0.0504261776804924
Train: 0.04134424030780792
Train: 0.053065139800310135
Train: 0.04002491384744644
Train: 0.048720814287662506
Train: 0.039852678775787354
Test: 0.5273698568344116
Test: 0.9336016774177551
Test: 0.9032295346260071
Test: 0.14981865882873535
Test: 0.9262117147445679
Test: 0.524742603302002
Test: 0.672063410282135
Train: 0.06154833734035492
Train: 0.045

Test: 0.5576072335243225
Test: 0.7259693145751953
Train: 0.040119465440511703
Train: 0.040102552622556686
Train: 0.044772662222385406
Train: 0.04313237592577934
Train: 0.035291410982608795
Train: 0.037453025579452515
Train: 0.037526894360780716
Train: 0.04365206137299538
Train: 0.03929319232702255
Train: 0.03604515641927719
Train: 0.02994343265891075
Train: 0.03178071975708008
Train: 0.03178738057613373
Train: 0.029130667448043823
Train: 0.04224664345383644
Train: 0.03177984058856964
Train: 0.03448488190770149
Train: 0.02590804547071457
Train: 0.030275272205471992
Train: 0.02602260373532772
Train: 0.04223833605647087
Train: 0.04087526723742485
Train: 0.034034375101327896
Train: 0.04619592800736427
Train: 0.036432355642318726
Train: 0.03820382058620453
Train: 0.03015507571399212
Test: 0.5185620784759521
Test: 0.9643530249595642
Test: 0.9416577816009521
Test: 0.0901002511382103
Test: 0.9656643867492676
Test: 0.508918285369873
Test: 0.6688687205314636
Train: 0.03856344521045685
Train: 0.0

Test: 0.6372933983802795
Test: 0.8033756613731384
Train: 0.0395023338496685
Train: 0.036071062088012695
Train: 0.04199811443686485
Train: 0.04257209226489067
Train: 0.035287048667669296
Train: 0.03633414953947067
Train: 0.03830542415380478
Train: 0.03931950777769089
Train: 0.03357474505901337
Train: 0.0327342264354229
Train: 0.03189350664615631
Train: 0.033275704830884933
Train: 0.028579259291291237
Train: 0.026307035237550735
Train: 0.03849205747246742
Train: 0.0267260130494833
Train: 0.029946021735668182
Train: 0.022138869389891624
Train: 0.02784312516450882
Train: 0.026632316410541534
Train: 0.03680667653679848
Train: 0.03382035344839096
Train: 0.030246062204241753
Train: 0.04233290255069733
Train: 0.030583450570702553
Train: 0.0352591872215271
Train: 0.027435842901468277
Test: 0.7255358695983887
Test: 1.2433403730392456
Test: 1.1781811714172363
Test: 0.28786399960517883
Test: 1.2159963846206665
Test: 0.7253648042678833
Test: 0.9246613383293152
Train: 0.040426235646009445
Train: 0.0

...

Train: 0.043203867971897125
Train: 0.038881815969944
Train: 0.03456461802124977
Train: 0.038663171231746674
Train: 0.03320890665054321
Train: 0.03276893496513367
Train: 0.033948518335819244
Test: 0.07174506783485413
Test: 0.062245339155197144
Test: 0.05737389624118805
Test: 0.06228718161582947
Test: 0.06102152541279793
Test: 0.06476573646068573
Test: 0.06273654103279114
Train: 0.13981862366199493
Train: 0.22916817665100098
Train: 0.2205057144165039
Train: 0.03897454962134361
Train: 0.21919286251068115
Train: 0.12424623221158981
Train: 0.1277030110359192
Train: 0.04018104076385498
Train: 0.04524621739983559
Train: 0.03961861506104469
Train: 0.04391159117221832
Train: 0.04126186668872833
Train: 0.04115331172943115
Train: 0.044679995626211166
Train: 0.046295128762722015
Train: 0.0441192165017128
Train: 0.04158603772521019
Train: 0.04196065664291382
Train: 0.04007258638739586
Train: 0.03578684478998184
Train: 0.03910005837678909
Train: 0.03696475923061371
Train: 0.03323671594262123
Train: 

Train: 0.03937889263033867
Train: 0.03371645137667656
Train: 0.03158849477767944
Train: 0.03216731920838356
Train: 0.028838418424129486
Train: 0.028860678896307945
Train: 0.02906009927392006
Test: 0.35910889506340027
Test: 0.4039466083049774
Test: 0.35771554708480835
Test: 0.4725213348865509
Test: 0.4074426293373108
Test: 0.34967494010925293
Test: 0.39175453782081604
Train: 0.14049671590328217
Train: 0.23426489531993866
Train: 0.2238190621137619
Train: 0.03378898277878761
Train: 0.22200164198875427
Train: 0.12274450808763504
Train: 0.12810929119586945
Train: 0.03928342089056969
Train: 0.03976277634501457
Train: 0.03744511678814888
Train: 0.03936208039522171
Train: 0.035017065703868866
Train: 0.03828703612089157
Train: 0.04229406267404556
Train: 0.042356882244348526
Train: 0.040268655866384506
Train: 0.03815978392958641
Train: 0.036371078342199326
Train: 0.03819876164197922
Train: 0.03368576616048813
Train: 0.03737042844295502
Train: 0.03361712023615837
Train: 0.03247399255633354
Train:

Train: 0.03315092995762825
Train: 0.034450676292181015
Train: 0.03171175345778465
Train: 0.03168541565537453
Train: 0.030179880559444427
Train: 0.0269928015768528
Train: 0.02798675373196602
Train: 0.030438987538218498
Test: 0.04935086891055107
Test: 0.048876043409109116
Test: 0.04446781054139137
Test: 0.04984132945537567
Test: 0.04801732674241066
Test: 0.050729215145111084
Test: 0.047105688601732254
Train: 0.13674357533454895
Train: 0.2258206009864807
Train: 0.21483640372753143
Train: 0.033324893563985825
Train: 0.21346983313560486
Train: 0.1211489737033844
Train: 0.12546156346797943
Train: 0.03341370075941086
Train: 0.03599785268306732
Train: 0.03872712329030037
Train: 0.04227254539728165
Train: 0.03831463307142258
Train: 0.03651545196771622
Train: 0.039210401475429535
Train: 0.03845323994755745
Train: 0.036936886608600616
Train: 0.03139372915029526
Train: 0.03438626229763031
Train: 0.03336700424551964
Train: 0.03282472491264343
Train: 0.03378971293568611
Train: 0.03136899694800377
Tr

Train: 0.03406958654522896
Train: 0.03195153549313545
Train: 0.030469844117760658
Train: 0.0324527882039547
Train: 0.031181829050183296
Train: 0.0292735006660223
Train: 0.02737019769847393
Train: 0.025163402780890465
Train: 0.025068268179893494
Train: 0.028648238629102707
Test: 0.08042734861373901
Test: 0.07954449206590652
Test: 0.07269611209630966
Test: 0.0805538073182106
Test: 0.08018450438976288
Test: 0.08497277647256851
Test: 0.07718338817358017
Train: 0.13288868963718414
Train: 0.22089041769504547
Train: 0.21359682083129883
Train: 0.03097712993621826
Train: 0.21050041913986206
Train: 0.11629834026098251
Train: 0.12281486392021179
Train: 0.03327779471874237
Train: 0.03789064288139343
Train: 0.03466465696692467
Train: 0.038262948393821716
Train: 0.037439506500959396
Train: 0.0360429622232914
Train: 0.03613483905792236
Train: 0.03861243277788162
Train: 0.037910547107458115
Train: 0.0329616479575634
Train: 0.034805331379175186
Train: 0.02942325547337532
Train: 0.029260914772748947
Tra

Train: 0.035447023808956146
Train: 0.0377480573952198
Train: 0.03494623303413391
Train: 0.030869614332914352
Train: 0.03397931531071663
Train: 0.030961036682128906
Train: 0.030281050130724907
Train: 0.03172944113612175
Train: 0.026567356660962105
Train: 0.027490006759762764
Train: 0.029187599197030067
Test: 0.05114177614450455
Test: 0.043548084795475006
Test: 0.0434153713285923
Test: 0.04548728093504906
Test: 0.04477667063474655
Test: 0.04878460615873337
Test: 0.04643191397190094
